In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from source import data_import
from source.calculate_mean_psd import calculate_mean_psd
from source.constants import CHANNELS, FREQUENCY_RANGES

print(f"loaded channel names: {CHANNELS}")

loaded channel names: ['F4-C4', 'F3-C3', 'FT9-FT10', 'FZ-CZ', 'F7-T7', 'FP2-F4', 'T8-P8-1', 'T8-P8-0', 'FP1-F3', 'CZ-PZ']


In [216]:
# load data
df_pp = data_import.load_pyarrow(file_name="preprocessed_seg_all_60")
df_pp.shape

/Users/anabroggini/Documents/bootcamp/ai2/source/../data/preprocessed_seg_all_60.arrow was loaded.


(6620160, 17)

In [217]:
test= df_pp.drop(['target'],axis=1)
missing_data = np.isnan(test)
missing_count = missing_data.sum()

# Display the count of missing values
print(missing_count)

channel
F4-C4             0
F3-C3             0
FT9-FT10          0
FZ-CZ             0
F7-T7             0
FP2-F4            0
T8-P8-1           0
T8-P8-0           0
FP1-F3            0
CZ-PZ             0
before_seizure    0
is_seizure        0
epoch             0
segment_id        0
seizure_start     0
seizure_id        0
dtype: int64


In [145]:
# df_pp = df_pp.iloc[:2_000_000,:]
# df_pp.shape

In [249]:
ids_column = df_pp.epoch

    # Find unique epoch IDs
unique_epoch_ids = ids_column.unique()
type(unique_epoch_ids)

numpy.ndarray

In [259]:
data=df_pp

In [261]:
ids_column = data.epoch

ids_column = data.epoch

# Find unique epoch IDs
unique_epoch_ids = ids_column.unique()
#unique_epoch_ids_np = unique_epoch_ids.to_numpy()

    # Shuffle the unique epoch IDs
np.random.seed(42)
np.random.shuffle(unique_epoch_ids)

    # Calculate the number of unique epoch IDs for training, validation, and testing
train_samples_nr = int(np.floor(0.8 * len(unique_epoch_ids)))

    # Split the unique epoch IDs into train, validation, and test sets
train_sample_ids = unique_epoch_ids[:train_samples_nr]
test_sample_ids = unique_epoch_ids[train_samples_nr:]

    # Create boolean masks for filtering
train_mask = ids_column.isin(train_sample_ids)
test_mask = ids_column.isin(test_sample_ids)

    # Apply the masks to create train and test samples
train_sample = data[train_mask]
test_sample = data[test_mask]

X_train = train_sample.drop(["is_seizure"], axis=1)
X_test = test_sample.drop(["is_seizure"], axis=1)

In [275]:
train_sample.tail(10)

channel,F4-C4,F3-C3,FT9-FT10,FZ-CZ,F7-T7,FP2-F4,T8-P8-1,T8-P8-0,FP1-F3,CZ-PZ,before_seizure,is_seizure,target,epoch,segment_id,seizure_start,seizure_id
0 days 00:30:59.960937500,-16.416032,-51.647151,33.964094,-219.782839,-40.663311,-63.639441,-7.349966,-7.349966,-189.775132,80.200202,False,False,0 days,429,59,False,0
0 days 00:30:59.964843750,-13.545649,-46.545615,32.684058,-213.368234,-38.645470,-65.302170,-1.513598,-1.513598,-190.307925,88.826809,False,False,0 days,429,59,False,0
0 days 00:30:59.968750,-11.051662,-45.659108,33.773493,-204.346879,-41.882482,-65.140890,1.254934,1.254934,-188.892028,95.589835,False,False,0 days,429,59,False,0
0 days 00:30:59.972656250,-9.037396,-43.472518,34.893408,-195.473583,-41.088917,-63.961173,0.144070,0.144070,-188.379799,102.090336,False,False,0 days,429,59,False,0
0 days 00:30:59.976562500,-5.121103,-38.569313,36.198546,-186.053438,-40.107826,-65.784596,-1.366541,-1.366541,-191.203823,108.881497,False,False,0 days,429,59,False,0
0 days 00:30:59.980468750,-0.864090,-32.603422,38.276695,-178.979398,-38.529120,-67.852734,-2.069763,-2.069763,-196.526483,115.471634,False,False,0 days,429,59,False,0
0 days 00:30:59.984375,0.280998,-20.070823,38.700630,-172.185136,-29.636411,-74.477263,-2.814388,-2.814388,-205.361929,124.743530,False,False,0 days,429,59,False,0
0 days 00:30:59.988281250,3.219222,-4.504176,37.070502,-164.234431,-16.655830,-79.180899,-1.146211,-1.146211,-213.432417,134.705479,False,False,0 days,429,59,False,0
0 days 00:30:59.992187500,5.378802,8.923870,38.367346,-156.512964,-9.254950,-80.109969,0.546591,0.546591,-218.495907,140.505550,False,False,0 days,429,59,False,0
0 days 00:30:59.996093750,3.883202,18.902130,38.577400,-149.544132,-6.236506,-81.719493,1.396617,1.396617,-223.136838,147.756717,False,False,0 days,429,59,False,0


In [297]:
def split_data(data, test_size=0.2, val_size=0.2, random_state=None):
    # Drop unnecessary features
    #data2 = data.drop(['before_seizure', 'epoch','segment_id','target'], axis =1)

    # Assuming the first column contains the IDs
    ids_column = data.epoch

    # Find unique epoch IDs
    unique_epoch_ids = ids_column.unique()
    #unique_epoch_ids_np = unique_epoch_ids.to_numpy()

    # Shuffle the unique epoch IDs
    np.random.seed(random_state)
    np.random.shuffle(unique_epoch_ids)

    # Calculate the number of unique epoch IDs for training, validation, and testing
    train_samples_nr = int(np.floor(0.8 * len(unique_epoch_ids)))

    # Split the unique epoch IDs into train, validation, and test sets
    train_sample_ids = unique_epoch_ids[:train_samples_nr]
    test_sample_ids = unique_epoch_ids[train_samples_nr:]

    # Create boolean masks for filtering
    train_mask = ids_column.isin(train_sample_ids)
    test_mask = ids_column.isin(test_sample_ids)

    # Apply the masks to create train and test samples
    train_sample = data[train_mask]
    test_sample = data[test_mask]

    X_train = train_sample.drop(["is_seizure"], axis=1)
    X_test = test_sample.drop(["is_seizure"], axis=1)

    Y_train = train_sample.iloc[:, -5]
    Y_test = test_sample.iloc[:, -5]

    #y_test = Y_test.groupby(ids_column[test_mask]).max()
    #y_train = Y_train.groupby(ids_column[train_mask]).max()

    return X_train, X_test, Y_train, Y_test #, y_train, y_test

In [298]:
X_train, X_test, Y_train, Y_test = split_data(df_pp, random_state=42)

In [278]:
print(f'train shape', Y_train.shape)
print(f'test shape', Y_test.shape)

train shape (5283840,)
test shape (1336320,)


In [295]:
Y_train.values

array([  0,   0,   0, ..., 429, 429, 429])

In [287]:
rows_per_segment_epoch = X_train.groupby(['epoch', 'segment_id']).size()
np.unique(rows_per_segment_epoch.values)

array([256])

In [288]:
rows_per_segment_epoch = X_test.groupby(['epoch', 'segment_id']).size()
np.unique(rows_per_segment_epoch.values)

array([256])

## Feature Extraction

In [289]:
# ignore for aggregation
ignore_col = []

# target definition
PRED_INTERVAL = 6000
target_colname = 'target'

In [290]:
### aggregate Functions for mean psd:
delta = lambda x: calculate_mean_psd(x, frequency_ranges={'Delta' : FREQUENCY_RANGES['Delta']})[x.name]['Delta']
theta = lambda x: calculate_mean_psd(x, frequency_ranges={'Theta' : FREQUENCY_RANGES['Theta']})[x.name]['Theta']
gamma = lambda x: calculate_mean_psd(x, frequency_ranges={'Gamma': FREQUENCY_RANGES['Gamma']})[x.name]['Gamma']

delta_agg = pd.NamedAgg(column='delta', aggfunc=delta)
theta_agg = pd.NamedAgg(column='theta', aggfunc=theta)
gamma_agg = pd.NamedAgg(column='gamma', aggfunc=gamma)

### aggregate Functions for target:
target_foo = lambda x, pred_interval=PRED_INTERVAL: 0 < x.dt.total_seconds().min() < pred_interval


In [291]:
abs_mean = lambda x: x.apply(abs).mean()
abs_mean_agg = pd.NamedAgg(column='abs_mean', aggfunc=abs_mean)

In [223]:
df_pp.head(100)

channel,F4-C4,F3-C3,FT9-FT10,FZ-CZ,F7-T7,FP2-F4,T8-P8-1,T8-P8-0,FP1-F3,CZ-PZ,before_seizure,is_seizure,target,epoch,segment_id,seizure_start,seizure_id
0 days 00:30:00,16.878726,20.334213,-3.322584,17.941953,18.739373,33.092935,12.625819,12.625819,44.788430,2.790970,False,False,0 days,0,0,False,0
0 days 00:30:00.003906250,22.891349,24.865283,-8.175141,23.784967,26.448730,40.639022,11.874291,11.874291,59.506243,4.538652,False,False,0 days,0,0,False,0
0 days 00:30:00.007812500,22.162881,25.360125,-8.902114,25.615925,27.023819,40.074853,5.285063,5.285063,60.136641,5.292160,False,False,0 days,0,0,False,0
0 days 00:30:00.011718750,29.573426,33.953147,-14.930178,33.038895,32.700204,52.956037,8.308854,8.308854,79.842157,8.558853,False,False,0 days,0,0,False,0
0 days 00:30:00.015625,21.292400,18.163275,-11.955401,21.366748,20.938086,35.328903,-1.132117,-1.132117,60.607725,6.096930,False,False,0 days,0,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 00:30:00.371093750,-34.796077,-22.800658,-35.635680,-45.915189,25.744824,-4.392587,-5.530252,-5.530252,21.714445,6.757269,False,False,0 days,0,0,False,0
0 days 00:30:00.375000,-33.543742,-21.933515,-28.799356,-49.717578,-1.499536,-27.902348,-16.265325,-16.265325,-40.542553,3.012091,False,False,0 days,0,0,False,0
0 days 00:30:00.378906250,-23.910864,-23.530688,-12.558581,-46.254773,-15.893778,-54.849923,-9.677097,-9.677097,-86.111333,-0.147218,False,False,0 days,0,0,False,0
0 days 00:30:00.382812500,-16.926063,-26.528119,-4.963009,-43.331668,-24.356403,-63.212030,-14.078722,-14.078722,-87.467561,1.709068,False,False,0 days,0,0,False,0


In [229]:
rows_per_segment_epoch = df_pp.groupby(['epoch', 'segment_id']).size()
np.unique(rows_per_segment_epoch.values)

array([256])

In [230]:
rows_per_segment_epoch

epoch  segment_id
0      0             256
       1             256
       2             256
       3             256
       4             256
                    ... 
430    55            256
       56            256
       57            256
       58            256
       59            256
Length: 25860, dtype: int64

In [195]:
df_features = df_pp.groupby(['epoch', 'segment_id']).agg(
    {C:['std',
        'var',
        #'mean',
        abs_mean_agg,
        delta_agg,
        theta_agg,
        gamma_agg
        ] for C in CHANNELS} | 
    {target_colname: [target_foo]} 
    ) 
df_features.head()

channel               F4-C4                                                \
                        std         var   abs_mean       delta      theta   
epoch segment_id                                                            
0     0           19.215291  369.227396  14.539727   39.509423  15.649456   
      1           19.292719  372.208995  15.081409   57.166882  13.481116   
      2           26.026211  677.363658  18.727514  112.603482  22.111449   
      3           26.999490  728.972443  18.877600  148.551723  15.161321   
      4           23.273241  541.643731  17.412866   77.372989  25.561364   

channel                         F3-C3                                     ...  \
                     gamma        std         var   abs_mean       delta  ...   
epoch segment_id                                                          ...   
0     0           1.143016  21.315638  454.356435  16.254065   50.669637  ...   
      1           1.218430  25.477037  649.079429  19.593329   43.699214  ...   
      2           0.943441  27.729263  768.912015  20.540334  114.429972  ...   
      3           1.178056  29.614908  877.042781  21.282515  160.101362  ...   
      4           1.423834  23.413506  548.192254  18.048609   58.495875  ...   

channel                FP1-F3                             CZ-PZ              \
                        delta       theta      gamma        std         var   
epoch segment_id                                                              
0     0            678.257264   92.822474  22.441816  16.368013  267.911860   
      1            228.536554   76.808250  33.427544  18.012047  324.433840   
      2           2277.403585  176.024973   8.914849  22.813508  520.456131   
      3           1990.870208   75.730101  21.215121  19.942794  397.715029   
      4           1174.757791  284.475569  16.303152  18.725786  350.655046   

channel                                                       target  
                   abs_mean      delta      theta     gamma <lambda>  
epoch segment_id                                                      
0     0           13.053429  33.181709  18.606278  0.272641    False  
      1           14.154003  40.078259  21.722061  0.268936    False  
      2           18.522101  78.066994  19.695134  0.256728    False  
      3           15.324033  60.340596  18.821070  0.258679    False  
      4           15.186356  46.083693  24.182717  0.271254    False  

[5 rows x 61 columns]

## Flatten Dataframe

In [196]:
# joining column names with agg functions, but leaving target column
df_features.columns = ['_'.join(col).strip() for col in df_features.columns.values if target_colname != col[0]] + [target_colname]
df_features.reset_index(inplace=True)
df_features.tail(10)


,epoch,segment_id,F4-C4_std,F4-C4_var,F4-C4_abs_mean,F4-C4_delta,F4-C4_theta,F4-C4_gamma,F3-C3_std,F3-C3_var,...,FP1-F3_delta,FP1-F3_theta,FP1-F3_gamma,CZ-PZ_std,CZ-PZ_var,CZ-PZ_abs_mean,CZ-PZ_delta,CZ-PZ_theta,CZ-PZ_gamma,target
25670,427,50,62.047396,3849.879395,48.532391,876.443336,47.293244,0.116680,48.487299,2351.018165,...,1005.431235,196.185074,0.130984,44.610666,1990.111533,35.290335,387.385715,116.689240,0.127599,False
25671,427,51,64.757171,4193.491187,52.397825,1001.668289,149.447265,0.152441,57.386984,3293.265936,...,1020.358622,375.249208,0.138803,76.142205,5797.635311,63.115132,1300.871869,179.537494,0.123257,False
25672,427,52,60.284399,3634.208821,47.856674,638.205156,83.567239,0.113904,57.080254,3258.155429,...,761.945608,267.638927,0.127543,89.348866,7983.219877,74.186671,1858.263056,228.617230,0.125107,False
25673,427,53,58.341042,3403.677188,47.065547,742.525596,138.505577,0.205220,68.153564,4644.908230,...,930.952618,236.217033,0.263344,72.347418,5234.148954,55.600307,1050.967162,263.790037,0.137042,False
25674,427,54,74.325070,5524.216008,56.468569,1175.387907,112.819480,0.136509,64.478043,4157.418032,...,601.521044,196.220678,0.129774,76.340844,5827.924501,59.213101,1156.770911,141.905120,0.132452,False
25675,427,55,68.358893,4672.938283,51.167982,1008.630285,141.046879,0.087317,66.606519,4436.428411,...,1179.674268,133.796785,0.147048,58.948054,3474.873057,47.315268,685.726762,194.627585,0.127762,False
25676,427,56,66.675303,4445.596052,55.899346,1062.932877,71.866556,0.096636,63.266861,4002.695695,...,722.282712,188.197257,0.149490,59.530137,3543.837173,48.161461,710.346626,128.530454,0.100497,False
25677,427,57,68.882581,4744.810028,57.070465,1066.701407,191.259247,0.114321,65.631443,4307.486300,...,1298.137313,256.822422,0.159794,67.711541,4584.852722,53.934476,824.942006,213.612137,0.106948,False
25678,427,58,70.045835,4906.418958,56.009773,1077.159557,195.166910,0.103390,69.186297,4786.743721,...,1696.708837,407.514293,0.152785,76.280400,5818.699392,57.788459,1093.099933,295.959929,0.123690,False
25679,427,59,58.728981,3449.093155,45.870640,770.418180,66.455653,0.117682,63.567165,4040.784412,...,525.210394,118.638727,0.126069,42.755134,1828.001480,34.592321,354.093066,91.083292,0.149015,False


In [197]:
true_counts_per_epoch = df_features[df_features['target'] == True].groupby('epoch')['target'].sum()
true_counts_per_epoch.shape

(57,)

In [299]:
df_features.shape

(25680, 63)

In [300]:
# #Preparing data to be scaled

num_features= df_features.drop(['epoch','segment_id','target'],axis =1)

from sklearn.preprocessing import StandardScaler, minmax_scale, MinMaxScaler

scaler = StandardScaler()
num_features_scaled = scaler.fit_transform(num_features)

/Users/anabroggini/Documents/bootcamp/ai2/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/anabroggini/Documents/bootcamp/ai2/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/anabroggini/Documents/bootcamp/ai2/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/anabroggini/Documents/bootcamp/ai2/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:767

In [301]:
original_array =np.array(df_features)
target=original_array[:,-1]
target = target[:, np.newaxis]
segseiz_column = original_array[:, 0:2] ## epoch and segment_id
array_all_scaled = np.concatenate((segseiz_column, num_features_scaled, target), axis=1)
array_all_scaled.shape

(25680, 63)

In [10]:
# # Determine the number of epochs
# num_epochs = df_features.epoch.nunique()
# # # Determine the number of segments for each epoch
# num_segments = df_features.segment_id.nunique()  # Assuming there are 30 segments for each epocha
# num_features = array_all_scaled.shape[1]

# reshaped_all= array_all_scaled.reshape(num_epochs, num_segments, num_features)
# reshaped_all.shape

(77, 30, 73)

In [304]:
num_segments

60

In [302]:
import numpy as np


original_array =np.array(df_features)

# Extract the epoch column
epoch_column = original_array[:, 0]

# Determine the number of epochs (assuming epochs are from 1 to number of segments)
num_epochs = df_features.epoch.unique()[-1]

# Determine the number of segments for each epoch
num_segments = len(df_features.segment_id.unique())  # Assuming there are 30 segments for each epoch

# Initialize an empty 3D array
reshaped_array = np.empty((num_epochs,num_segments, array_all_scaled.shape[1]))

# Reshape the data for each epoch and insert it into the 3D array
for epoch in range(num_epochs):
    start_idx = epoch * num_segments
    end_idx = (epoch + 1) * num_segments
    reshaped_array[epoch,:, :] = array_all_scaled[start_idx:end_idx,:]

In [201]:
reshaped_array.shape

(427, 60, 63)

In [202]:
import numpy as np

# Assuming original_data_array has dimensions (batch, sequence, features)

num_batches, num_sequences, num_features = reshaped_array.shape

# Define the window size
window_size = 30

# Calculate the number of augmented batches
num_augmented_batches = num_sequences - window_size + 1

# Create an empty array for the augmented data
augmented_data_array = np.zeros((num_batches * num_augmented_batches, window_size, num_features))

# Iterate through batches
for batch_idx in range(num_batches):
    # Iterate through sequences to create augmented batches
    for seq_idx in range(num_augmented_batches):
        # Copy the window of data
        augmented_data_array[batch_idx * num_augmented_batches + seq_idx, :, :] = reshaped_array[batch_idx, seq_idx:seq_idx + window_size, :]

# The resulting augmented_data_array will have the desired structure



In [203]:
augmented_data_array.shape

(13237, 30, 63)

In [204]:
target= augmented_data_array[:,:,-1]
target.shape

target.max(axis=1).sum()


1767.0

In [205]:

np.save('data/feature_extract_reshaped_60_5.npy', augmented_data_array)

In [26]:
# from source.data_import import save_pyarrow

# save_pyarrow(df_features, file_name="feature_extracted")

/Users/anabroggini/Documents/bootcamp/ai2/source/../data/feature_extracted.arrow was successfully written.
